In [ ]:
!pip install natasha
!pip install spacy
!python -m spacy download ru_core_news_sm

### Пункт1

Я взял отрывок стихотворения из романа Набокова. Во-первых, я хочу проверить насколько хорошо теггеры справляются с различением имен собственных и просто слов, написанных с большой буквы. Во-вторых, Набоков обильно поставляет сложные слова, редкие слова, окказионализмы. Также тут много местоимений, причастий, деепричастий, частиц, союзов, которые могут представлять сложности с точки зрения выбора подхода

In [14]:
text = """
Я тень, я свиристель, убитый влет
Подложной синью, взятой в переплет
Окна; комочек пепла, легкий прах,
Порхнувший в отраженных небесах.
Так и снутри удвоены во мне
Я сам, тарелка, яблоко на ней;
Раздвинув ночью шторы, за стеклом
Я открываю кресло со столом,
Висящие над темной гладью сада,

Но лучше, если после снегопада
Они, как на ковре, стоят вовне --
Там, на снегу, в хрустальнейшей стране!

Вернемся в снегопад: здесь каждый клок
Бесформен, медлен, вял и одинок.
Унылый мрак, белесый бледный день,
Нейтральный свет, абстрактных сосен сень.
В ограду сини вкрадчиво-скользящей
Ночь заключит картину со смотрящим;
А утром -- чьи пришпоренные ноги

Вписали строчку в чистый лист дороги? --
Дивится перл мороза. Снова мы
Направо слева ясный шифр зимы
Читаем: точка, стрелка вспять, штришок,
Вновь точка, стрелка вспять... фазаний скок!
Се гордый граус, родственник тетерки
Китаем наши претворил задворки.
Из "Хольмса", что ли: вспять уводит след,
Когда башмак назад носком надет.

Был люб мне, взоры грея, всякий цвет.

Я мог сфотографировать предмет
В своем зрачке. Довольно было мне
Глазам дать волю или, в тишине,
Шепнуть приказ, -- и все, что видит взор, --
Паркет, гикори лиственный убор,
Застрех, капели стылые стилеты
На дне глазницы оседало где-то
И сохранялось час, и два. Пока
Все это длилось, стоило слегка
Прикрыть глаза -- и заново узришь
"""

In [17]:
import re
text = re.sub(r'[^\w\s]', '', text)
#я вырезал знаки препинания для удобства

In [106]:
from nltk.tokenize import word_tokenize
tokens = word_tokenize(text)
tokens[88] = 'вкрадчиво-скользящей'
tokens[189] = 'где-то'
#вернул дефисы словам, потерявшим его

In [37]:
import csv
with open('corpora.csv', "w") as f:
    writer = csv.DictWriter(f, fieldnames = ["tokens", "POS"], delimiter = ',')
    writer.writeheader()
    for token in tokens:
        f.write(token + '\n')
#записываю в csv табличку для удобства размечания

In [108]:
filename = open('corpora.csv', 'r')
file = csv.DictReader(filename)
pos_gold = []
for col in file:
    pos_gold.append(col['POS'])
#достаю список с размеченными тегами

### Пункт2

In [129]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

pos_pymorphy = []

for token in tokens:
    ana = morph.parse(token)
    first = ana[0]
    pos_pymorphy.append(first.tag.POS)

In [122]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsMorphTagger,
    Doc
)

segmenter = Segmenter()

morph_tagger = NewsMorphTagger(emb)

In [123]:
pos_natasha = []

doc = Doc(' '.join(tokens))
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for token in doc.tokens:
    pos_natasha.append(token.pos)

In [176]:
import spacy

pos_spacy = []

nlp = spacy.load("ru_core_news_sm")

text = ' '.join(tokens)
doc = nlp(text)
for token in doc:
    #spacy не самым приятным способом работает с дефисом, поэтому вручную делаю, чтобы не было 3 сущностей на 1 токен
    if token.text not in ['-', 'то', 'вкрадчиво']:
        pos_spacy.append(token.pos_)

### Пункт3

Я сам делал разметку в соответствии с принципами UD, я уже работал с ним раньше, он кажется мне простым и удобным, только все PART отнес к ADV, мне кажется так сильно проще, а разделение не принципиально. Поэтому я привожу PART к ADV для наташи и стэйси, т.к. они тоже основаны на UD, а для пайморфи проделываю более основательную переработку. Также я решил объединить союзы в conj, т.к. выудить разделение из pymorphy не представляется возможным

In [179]:
for i in range(len(pos_spacy)):
    if pos_spacy[i] == 'PART':
        pos_spacy[i] = 'ADV'
    if pos_spacy[i] in ['SCONJ', 'CCONJ']:
        pos_spacy[i] = 'CONJ'
    if pos_natasha[i] == 'PART':
        pos_natasha[i] = 'ADV'
    if pos_natasha[i] in ['SCONJ', 'CCONJ']:
        pos_natasha[i] = 'CONJ'
    if pos_pymorphy[i] in ['ADVB', 'PRCL']:
        pos_pymorphy[i] = 'ADV'
    if pos_pymorphy[i] in ['INFN', 'GRND', 'PRTF', 'PRTS']:
        pos_pymorphy[i] = 'VERB'
    if pos_pymorphy[i] == 'NUMR':
        pos_pymorphy[i] = 'NUM'
    if pos_pymorphy[i] == 'PREP':
        pos_pymorphy[i] = 'ADP'
    if pos_pymorphy[i] in ['ADJF', 'ADJS']:
        pos_pymorphy[i] = 'ADJ'
    if pos_pymorphy[i] == 'NPRO':
        pos_pymorphy[i] = 'PRON'
# COMP нет желания как-то адаптировать, просто неадекватный подход

In [180]:
from sklearn.metrics import accuracy_score
print('accuracy_pypormphy:', accuracy_score(pos_gold, pos_pymorphy))
print('accuracy_natasha:', accuracy_score(pos_gold, pos_natasha))
print('accuracy_spacy:', accuracy_score(pos_gold, pos_spacy))

accuracy_pypormphy: 0.8009708737864077
accuracy_natasha: 0.7330097087378641
accuracy_spacy: 0.7912621359223301


На удивление лучший результат показывает pymorphy (это при том, что мы раскрывали омонимию по первому варианту), хотя, предварительно разглядывая, я ожидал в победителях spacy, возможно, некоторое упрощение его(пайморфи) более разветвленной разметки сыграло на руку, но изначально большая точность как будто заслуживает такого исхода. Ключевую разницу сыграли здесь слова с большой буквы: наташа постоянно пыталась отнести их к PROPN, поэтому ее результат заметно хуже. Точнее всего с ними работал как раз pymormphy, также часто pymorphy обильно использовал ADV, иногда попадая точнее других моделей, иногда, напротив, ошибаясь. Мне кажется релевантным выделение DET, и, хотя pymorphy не использовала этот тег, это ей не помешало занять первое место (если пойти навстречу ей в этом вопрсое точность окажется очень высокой). Выполним следующий шанс с pymorphy. Учитывая сложность текста, кажется, все модели справились достойно

### Пункт4

я хочу взять следующие 3 шаблона: ADJ + игра, ADJ + баланс, ADV + играть. Мне кажется, что биграммы таких шаблонов частотны в отзывах всех типов, при этом достаточно однозначно характеризуют отношение к игре, отдельно же данные прилагательные могут встречаться в отзывах обеих тональностей, в результате чего, скорее всего, самоуничтожаются, отдельно же существительные так же самоуничтожаются и не несут какого-либо положительного или отрицательного отношения.

Возьмем часть кода, в которой я записывал слова по словарям и встроим в нее чанкер. Изначально она выглядит так:

In [ ]:
for i in range(len(X_train)):
    if y_train[i] == 'pos':
        for token in X_train[i]:
            if token not in dict_pos.keys():
                dict_pos[token] = 1
            else:
                dict_pos[token] += 1
    else:
        for token in X_train[i]:
            if token not in dict_neg.keys():
                dict_neg[token] = 1
            else:
                dict_neg[token] += 1

После интеграции чанкера получаем:

In [ ]:
for i in range(len(X_train)):
    if y_train[i] == 'pos':
        for x, token in enumerate(X_train[i]):
            if token not in dict_pos.keys():
                dict_pos[token] = 1
            else:
                dict_pos[token] += 1
            if token in ['игра', 'баланс'] and x != 0:
                #смотрим на предыдущее слово, вдруг оно ADJ
                ana = morph.parse(X_train[i][x-1])
                first = ana[0]
                if first.tag.POS == 'ADJF':
                    bigram = X_train[i][x-1] + ' ' + X_train[i][x]
                    if bigram not in dict_pos.keys():
                        dict_pos[bigram] = 1
                    else:
                        dict_pos[bigram] += 1
            elif token == 'играть' and x != 0:
                #смотрим на предыдущее слово, вдруг оно ADV
                ana = morph.parse(X_train[i][x-1])
                first = ana[0]
                if first.tag.POS == 'ADVB':
                    bigram = X_train[i][x-1] + ' ' + X_train[i][x]
                    if bigram not in dict_pos.keys():
                        dict_pos[bigram] = 1
                    else:
                        dict_pos[bigram] += 1
                
    else:
        for x, token in enumerate(X_train[i]):
            if token not in dict_neg.keys():
                dict_neg[token] = 1
            else:
                dict_neg[token] += 1
            if token in ['игра', 'баланс'] and x != 0:
                #смотрим на предыдущее слово, вдруг оно ADJ
                ana = morph.parse(X_train[i][x-1])
                first = ana[0]
                if first.tag.POS == 'ADJF':
                    bigram = X_train[i][x-1] + ' ' + X_train[i][x]
                    if bigram not in dict_neg.keys():
                        dict_neg[bigram] = 1
                    else:
                        dict_neg[bigram] += 1
            elif token == 'играть' and x != 0:
                #смотрим на предыдущее слово, вдруг оно ADV
                ana = morph.parse(X_train[i][x-1])
                first = ana[0]
                if first.tag.POS == 'ADVB':
                    bigram = X_train[i][x-1] + ' ' + X_train[i][x]
                    if bigram not in dict_neg.keys():
                        dict_neg[bigram] = 1
                    else:
                        dict_neg[bigram] += 1

на деле точность не меняется. список положительных слов полезно пополнился, а вот список негативных не пополняется по старой проблеме: среди положительных отзывов много на самом деле отрицательных отзывов с хорошей оценкой. вероятно, на большей выборке результат появится, если среди положительных отзывов не будет негативных текстов, то он значительно улучшится. Я убежден, что теги валидно подобраны. Хуже результат от них точно не станет. Еще, вероятно, биграммам сложно пробиться через барьер >2/1 употребления на таком маленьком количестве текстов